In [ ]:
!git clone https://github.com/ozyygen/predict-KGE-TPR.git

In [ ]:
import os
os.chdir("/content/predict-KGE-TPR")

In [ ]:
%%capture
!pip install obonet

In [ ]:
import pandas
import src.utils
from src.models import TransE
from src.model_utils import load_goa_files, generate_GOA_valid_triplets, generate_GOA_train_triplets

In [ ]:
code2rel = dict({'F' :'http://purl.obolibrary.org/obo/GO_0003674',
                 'P': 'http://purl.obolibrary.org/obo/GO_0008150',
                 'C' :'http://purl.obolibrary.org/obo/GO_0005575'})

In [ ]:
spec_version_v1    = "2017-01-01"
spec_version_v2 = "2018-01-01"
dataset_date = "17-18"

In [ ]:
#get 2 GO human annotation version from 2017 to current (2022) each time. Now  "2017-01-01" and "2018-01-01"
#first version will be used for train set
triple_df = src.utils.generate_goa_graph("/your_GO_path","/your_GOA_path", spec_version_v1,1)
triple_df.rename({0: 'head', 1: 'relation', 2:'tail'}, axis=1, inplace=True)
triple_df.head()

In [ ]:
#define diff for test and valid sets
#diff between first and second versions will be used for test and valid set
#test and valid sets contain data that follows TPR

triple_df = src.utils.generate_goa_graph("/your_GO_path","/your_GOA_path",spec_version_v1,0)
second_df = src.utils.generate_goa_graph("/your_GO_path","/your_GOA_path",spec_version_v2,0)
#read gene ontology
graph_go = src.utils.read_gene_ontology("/your_GO_path"+spec_version_v1)
#take entities in V1
entities_v1 = src.utils.get_entities(triple_df)

#add level 1 train set entity implicit annotations ( hierarchical level 1) into test set
level1_data = set()
#inheriting the second-level ancestry functions.
final_sc2_2 = set()

diffv2_v1 = second_df.merge(
    triple_df, how='outer', indicator=True
).query('_merge == "left_only"').drop('_merge', 1)
level2_data = src.utils.add_level2_semantic(graph_go, diffv2_v1, entities_v1, spec_version_v1)
final_sc2_2 = src.utils.final_test_valid_sc2(triple_df,level2_data,graph_go)
test, valid = src.utils.split_test_valid(final_sc2_2)
#generate txt files
src.utils.txt_gen('test.txt',test)
src.utils.txt_gen('valid.txt',valid)

In [ ]:
#add TPR-inferred implicit annotation to train set
valid_df = pandas.read_csv('valid.txt',
                      header=None, 
                      names=["head","relation","tail"], 
                      delimiter="\t")
triple_df = pandas.concat([triple_df,valid_df]).drop_duplicates().reset_index(drop=True)

In [ ]:
#save train set 
triple_df.to_csv('train.txt', sep='\t', index=False,header = False)